In [1]:
import os 
cwd = os.getcwd()
cwd + "\\test.html"

'c:\\Users\\张晓鹏\\Desktop\\新建文件夹\\web_driver\\test.html'

## Methods dev chunk

In [32]:
import time
import random
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from PIL import Image

def setup_driver():
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/89.0",
    ]
    window_sizes = ["1920x1080", "1366x768", "1280x720"]

    options = Options()
    options.add_argument(f'user-agent={random.choice(user_agents)}')
    options.add_argument(f'--window-size={random.choice(window_sizes)}')
    # options.add_argument('--headless')  # Run in headless mode if required

    driver = webdriver.Chrome(options=options)
    return driver

def get_html(url):
    driver = setup_driver()
    try:
        driver.get(url)
        # Wait 10 seconds so that the content can be fully loaded
        wait = WebDriverWait(driver, 20)
        wait.until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
        # Mimicking human behavior by waiting a random time
        time.sleep(random.randint(2,5))
        html = driver.page_source

    finally:
        driver.quit()
    return html

def get_captcha(html, id):
    soup = BeautifulSoup(html, 'html.parser')
    captcha_element = soup.find(id=id)
    pretty_html = soup.prettify()
    captcha_url = captcha_element['src']
    captcha_url = captcha_url[5:]
    image_response = requests.get(captcha_url)
    if image_response.status_code == 200:
        with open("downloaded_image.png", "wb") as file:
            file.write(image_response.content)
        print("Image saved as downloaded_image.png")
    else:
        print("Failed to download the image")
    # Specify the path and name of your HTML file
    file_path = cwd + '\\test.png'
    # Open a file at the specified path in 'write' mode
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(pretty_html)  # Write the JavaScript code to the file




def get_captcha(url):
    cwd = os.getcwd()
    screenshot_path = cwd + "\\full_screenshot.png"
    driver = setup_driver()
    try:
        driver.get(url)
        # Wait 10 seconds so that the content can be fully loaded
        wait = WebDriverWait(driver, 20)
        wait.until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
        # Mimicking human behavior by waiting a random time
        time.sleep(random.randint(2,5))
        # Scroll to a specific position (e.g., 500 pixels down)
        scroll_position = 1000  # Replace with the desired vertical scroll position in pixels
        driver.execute_script(f"window.scrollTo(0, {scroll_position});")
        time.sleep(2)  # Adjust sleep time if needed for page loading
        driver.save_screenshot("full_screenshot.png")
        with Image.open(screenshot_path) as img:
            width, height = img.size
    finally:
        driver.quit()
    return width, height

JavaScript file written successfully!


In [ ]:
# Test get_captcha function
url = 'https://www.rdv-prefecture.interieur.gouv.fr/rdvpref/reservation/demarche/4443/cgu/'

width, height = get_captcha(url)

cw, ch = width/2, height/2
r = 0.15

with Image.open("full_screenshot.png") as full_img:
    fine_crop = 140 # Fine-tune the cropping by adjusting this value
    fine_crop_right = 100 # Fine-tune the cropping by adjusting this value
    # (left, top, right, bottom)
    element_screenshot = full_img.crop((cw-r*width, ch-r*height+fine_crop, cw+r*width-fine_crop_right, ch+r*height-fine_crop))
    element_screenshot.save("element_screenshot.png")  # Save the cropped image

print("JavaScript file written successfully!")

## Get captchas loop

In [ ]:
num_captchas = 2
for i in range(num_captchas):
    url = 'https://www.rdv-prefecture.interieur.gouv.fr/rdvpref/reservation/demarche/4443/cgu/'

    width, height = get_captcha(url)

    cw, ch = width/2, height/2
    r = 0.15

    with Image.open("full_screenshot.png") as full_img:
        fine_crop = 140 # Fine-tune the cropping by adjusting this value
        fine_crop_right = 100 # Fine-tune the cropping by adjusting this value
        # (left, top, right, bottom)
        element_screenshot = full_img.crop((cw-r*width, ch-r*height+fine_crop, cw+r*width-fine_crop_right, ch+r*height-fine_crop))
        element_screenshot.save(f"captchas/screenshot_{i}.png")  # Save the cropped image

    print(f"JavaScript file written successfully {i+1} times!")

## ViT Model

In [7]:
from transformers import DonutProcessor, VisionEncoderDecoderModel, AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from PIL import Image

# Load Donut model and processor
model_name = "naver-clova-ix/donut-base-finetuned-docvqa"  # or another fine-tuned model
processor = DonutProcessor.from_pretrained(model_name,
                                           cache_dir="./cache",  # Specify a cache directory
                                           force_download=False,  # Force re-download of the model files
                                           local_files_only=True)  # Ensure it tries to download if not available locally)
model = VisionEncoderDecoderModel.from_pretrained(model_name)

# Move model to device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Config of the encoder: <class 'transformers.models.donut.modeling_donut_swin.DonutSwinModel'> is overwritten by shared encoder config: DonutSwinConfig {
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    14,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": [
    2560,
    1920
  ],
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "donut-swin",
  "num_channels": 3,
  "num_heads": [
    4,
    8,
    16,
    32
  ],
  "num_layers": 4,
  "patch_size": 4,
  "path_norm": true,
  "qkv_bias": true,
  "transformers_version": "4.46.0",
  "use_absolute_embeddings": false,
  "window_size": 10
}

Config of the decoder: <class 'transformers.models.mbart.modeling_mbart.MBartForCausalLM'> is overwritten by shared decoder config: MBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "add

VisionEncoderDecoderModel(
  (encoder): DonutSwinModel(
    (embeddings): DonutSwinEmbeddings(
      (patch_embeddings): DonutSwinPatchEmbeddings(
        (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DonutSwinEncoder(
      (layers): ModuleList(
        (0): DonutSwinStage(
          (blocks): ModuleList(
            (0-1): 2 x DonutSwinLayer(
              (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attention): DonutSwinAttention(
                (self): DonutSwinSelfAttention(
                  (query): Linear(in_features=128, out_features=128, bias=True)
                  (key): Linear(in_features=128, out_features=128, bias=True)
                  (value): Linear(in_features=128, out_features=128, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
           

In [18]:
# Load an image from file
#image_path = "captchas/screenshot_1.png"
image_path = "captchas/simple.jpg"
image = Image.open(image_path).convert("RGB")

# Preprocess the image for the model
pixel_values = processor(images=image, return_tensors="pt").pixel_values
pixel_values = pixel_values.to(device)
pixel_values.shape

torch.Size([1, 3, 2560, 1920])

In [23]:
# Generate text
with torch.no_grad():
    outputs = model.generate(pixel_values, max_length=8, min_length=6, num_beams=5)

# Decode the generated text
recognized_text = processor.decode(outputs[0], skip_special_tokens=True)

print(f"Recognized Text: {recognized_text}")


Recognized Text: m m m m m m


### This actually works very well!!!

In [25]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch

# Load the pre-trained TrOCR model and processor
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.46.0"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder

In [29]:
# Load and preprocess the image
image = Image.open("captchas/screenshot_1.png").convert("RGB")
pixel_values = processor(images=image, return_tensors="pt").pixel_values

# Perform OCR
outputs = model.generate(pixel_values)
recognized_text = processor.batch_decode(outputs, skip_special_tokens=True)[0]
print("Recognized Text:", recognized_text)

Recognized Text: PUSKIT


## Putting it all together

In [30]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch

# Load the pre-trained TrOCR model and processor
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.46.0"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder

In [35]:
num_captchas = 20
for i in range(num_captchas):
    url = 'https://www.rdv-prefecture.interieur.gouv.fr/rdvpref/reservation/demarche/4443/cgu/'

    width, height = get_captcha(url)

    cw, ch = width/2, height/2
    r = 0.15

    with Image.open("full_screenshot.png") as full_img:
        fine_crop = 140 # Fine-tune the cropping by adjusting this value
        fine_crop_right = 100 # Fine-tune the cropping by adjusting this value
        # (left, top, right, bottom)
        element_screenshot = full_img.crop((cw-r*width, ch-r*height+fine_crop, cw+r*width-fine_crop_right, ch+r*height-fine_crop))
        element_screenshot.save(f"captchas/screenshot_{i+1}.png")  # Save the cropped image
    image_path = f"captchas/screenshot_{i+1}.png"
    image = Image.open(image_path).convert("RGB")
    pixel_values = processor(images=image, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    with torch.no_grad():
        outputs = model.generate(pixel_values, max_length=8, min_length=6, num_beams=5)

    recognized_text = processor.decode(outputs[0], skip_special_tokens=True)
    print(f"Recognized Text {i+1}: {recognized_text}")

    print(f"JavaScript file written successfully {i+1} times!")

Recognized Text 1: NUY8MSXTV
JavaScript file written successfully 1 times!
Recognized Text 2: JIKM6RH
JavaScript file written successfully 2 times!
Recognized Text 3: UNEASSA I
JavaScript file written successfully 3 times!
Recognized Text 4: 639DWA4
JavaScript file written successfully 4 times!
Recognized Text 5: P6T9SPUE
JavaScript file written successfully 5 times!
Recognized Text 6: RPRBEHSX
JavaScript file written successfully 6 times!
Recognized Text 7: ENDKNAFFE
JavaScript file written successfully 7 times!
Recognized Text 8: 81899563-
JavaScript file written successfully 8 times!
Recognized Text 9: +8MNRS-
JavaScript file written successfully 9 times!
Recognized Text 10: MYBNA30B
JavaScript file written successfully 10 times!
Recognized Text 11: INVEDVICE!!
JavaScript file written successfully 11 times!
Recognized Text 12: X3WEA6
JavaScript file written successfully 12 times!
Recognized Text 13: DEXPAXKY
JavaScript file written successfully 13 times!
Recognized Text 14: SYSY5S
J

KeyboardInterrupt: 